# Transformer all in one

In [48]:
%%capture
%load_ext autoreload
%autoreload 2

## Imports

In [49]:
import transformer_all
from transformer_all import Config, TrainHandler, DataHandler, Modules, Format

In [50]:
format = Format(style="solarized-light")
print_py = format.print_py
print_obj = format.print_obj
print_src = format.print_src

In [51]:
config=Config()
print_obj(config)

Config(seed=0,
       preview=slice(None, 10, None),
       example_item=1337,
       max_length=512,
       min_freq=5,
       default_token='<unk>',
       special_tokens=['<unk>', '<pad>'],
       batch_size=512,
       lr=0.001,
       num_epochs=20,
       heads=4,
       emb=32,
       device='cuda',
       start_time=1710127576.6947355,
       last_time=0.0)



In [52]:
train_handler = TrainHandler(config=config)
print_src(train_handler.__init__)

    def __init__(self, config: Config):
        self.config = config
        np.random.seed(self.config.seed)
        torch.manual_seed(self.config.seed)
        torch.cuda.manual_seed(self.config.seed)
        torch.backends.cudnn.deterministic = True



In [53]:
data_handler = DataHandler(config=config)
print_src(data_handler.__init__)

  0.114225s   0.000000s |	DataHandler Start
  1.679308s   1.565083s |	DataHandler load_dataset
  1.679308s   0.000000s |	DataHandler get_tokenizer


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

  7.512112s   5.832805s |	DataHandler tokenize_all
 10.297424s   2.785312s |	DataHandler set_vocab


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

 21.075969s  10.778545s |	DataHandler numericalize_all
 21.076970s   0.001001s |	DataHandler format_all
 21.076970s   0.000000s |	DataHandler set_data_loaders
    def __init__(self, config: Config):
        self.config = config
        self.config.elapsed("DataHandler Start")
        self.dataset = datasets.load_dataset("zapsdcn/imdb", cache_dir="../data/classification/")
        self.config.elapsed("DataHandler load_dataset")
        self.train_dataset = self.dataset['train'] 
        self.validation_dataset = self.dataset['validation']
        self.test_dataset = self.dataset['test']
        self.tokenizer = torchtext.data.utils.get_tokenizer("basic_english")
        self.config.elapsed("DataHandler get_tokenizer")
        self.trim = slice(None,config.max_length,None)
        self.tokenize_all()
        self.config.elapsed("DataHandler tokenize_all")
        self.set_vocab()
        self.config.elapsed("DataHandler set_vocab")
        self.numericalize_all()
        self.config.elap

In [54]:
train_handler.load(data_handler=data_handler)
print_src(train_handler.load)

 21.173270s   0.096299s |	TrainHandler start
 21.179270s   0.006000s |	TrainHandler load
    def load(self, data_handler: DataHandler):
        self.config.elapsed("TrainHandler start")
        self.data_handler = data_handler
        self.model = Modules.Transformer(config=self.config, vocab_size=len(self.data_handler.vocab)).to("cuda")
        self.opt = torch.optim.Adam(lr=self.config.lr, params=self.model.parameters())
        self.config.elapsed("TrainHandler load")



In [55]:
print_src(Modules.Transformer)

    class Transformer(torch.nn.Module):
        def __init__(self, config: Config, vocab_size: int):
            super().__init__()
            self.config = config
            self.token_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=vocab_size)
            self.pos_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=self.config.max_length)
            self.tblock = Modules.TransformerBlock(config=self.config)
            self.toprobs = torch.nn.Linear(self.config.emb, 2)

        def forward(self, x):
            tokens = self.token_embedding(x)
            b, t, e = tokens.shape
            positions = self.pos_embedding(torch.arange(t, device="cuda"))[None, :, :].expand(b, t, e)
            x = tokens + positions
            x, W = self.tblock(x)
            x = torch.mean(x, dim=1)
            x = self.toprobs(x)
            return F.log_softmax(x, dim=1), W



In [56]:
train_handler.train()

 21.293531s   0.114261s |	train start
Epoch:0; Loss: 0.686713457107544; Validation Accuracy: 0.5042


In [ ]:
train_handler.save()